In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../Data/Split_highway.csv')

In [3]:
df.head()

,bicycle,bridge,cycleway,footway,highway,lanes,lit,maxspeed,name,oneway,...,id,version,tags,osm_type,fid,id_2,left,top,right,bottom
0,NaN,NaN,NaN,NaN,residential,NaN,NaN,NaN,Macondray Lane,NaN,...,5004035,-1,NaN,way,9789,9789,-122.417536,37.799515,-122.416396,37.798612
1,no,NaN,NaN,NaN,motorway,5.0,NaN,55 mph,James Lick Freeway,yes,...,5071582,-1,"{""NHS"":""STRAHNET"",""hgv"":""designated"",""hgv:nati...",way,11603,11603,-122.400434,37.719094,-122.399294,37.718191
2,no,NaN,NaN,NaN,motorway,5.0,NaN,55 mph,James Lick Freeway,yes,...,5071582,-1,"{""NHS"":""STRAHNET"",""hgv"":""designated"",""hgv:nati...",way,11718,11718,-122.399294,37.719094,-122.398154,37.718191
3,no,NaN,NaN,NaN,motorway,5.0,NaN,55 mph,James Lick Freeway,yes,...,5071582,-1,"{""NHS"":""STRAHNET"",""hgv"":""designated"",""hgv:nati...",way,11719,11719,-122.399294,37.718191,-122.398154,37.717287
4,no,NaN,NaN,NaN,motorway,5.0,NaN,55 mph,James Lick Freeway,yes,...,5071582,-1,"{""NHS"":""STRAHNET"",""hgv"":""designated"",""hgv:nati...",way,11720,11720,-122.399294,37.717287,-122.398154,37.716383


In [4]:
df.columns

Index(['bicycle', 'bridge', 'cycleway', 'footway', 'highway', 'lanes', 'lit',
       'maxspeed', 'name', 'oneway', 'ref', 'service', 'sidewalk', 'surface',
       'id', 'version', 'tags', 'osm_type', 'fid', 'id_2', 'left', 'top',
       'right', 'bottom'],
      dtype='object')

In [ ]:
df = df.drop(['version', 'id_2'], axis=1)

In [ ]:
df['tags'].value_counts()

In [ ]:
groups = df.groupby('fid')

In [ ]:
tile_value_count_df = df['fid'].value_counts().sort_values().to_frame().reset_index().rename(columns={'index': 'tile_id','fid':'count'})

In [ ]:
tile_value_count_df[tile_value_count_df['count']==1]['tile_id']

In [ ]:
temp_tile_count_one = df[df['fid'].isin(tile_value_count_df[tile_value_count_df['count']==1]['tile_id'])]

In [ ]:
temp_tile_count_one

In [ ]:
temp_tile_count_one.columns
for col in temp_tile_count_one.columns:
    print(f"{col}: {temp_tile_count_one[col].isnull().values.all()}, \
    {temp_tile_count_one[col].isnull().mean() * 100}")


In [ ]:
df

In [ ]:
df[df['fid']==10725]['bicycle'].value_counts()

In [ ]:
df['fid'].value_counts()

In [ ]:
for col in df.columns:
    print(col)
    print(df[col].value_counts(dropna=False,normalize=True))
    print("\n\n\n")
# df['fid'].value_counts()

## ROAD VECTORS TILED 

In [239]:
df = pd.read_csv('../Data/Road_Vectors_Tiled.csv')
df = df.rename(columns={"id": "Tile_ID"})

In [240]:
df.head()
print(df['RTTYP'].value_counts(dropna=False,normalize=False))


M      25271
NaN     1193
I        423
S        357
U        286
O         11
Name: RTTYP, dtype: int64


In [241]:
import random

sampleList = ['M', 'I', 'S', 'U', 'O']

randomList = random.choices(
sampleList, weights=(1, 0.01673, 0.014126, 0.011317, 0.000435))
list_of_randoms = []
for x in range(1193):
    randomList = random.choices(
    sampleList, weights=(1, 0.01673, 0.014126, 0.011317, 0.000435))
    list_of_randoms.append(randomList[0])
    
print(len(list_of_randoms))

1193


In [242]:
def mapper(x):
    if x in ['M','I','S','U','O']:
        return x
    else :
        return list_of_randoms.pop()
df['RTTYP']=df['RTTYP'].apply(mapper)
df['RTTYP'].value_counts(dropna=False)

M    26412
I      441
S      376
U      301
O       11
Name: RTTYP, dtype: int64

In [243]:
# count_roadtype_per_tile = df.Tile_ID.value_counts().reset_index().\
#         rename(columns={'Tile_ID':'count','index':'Tile_ID'})

# count_roadtype_per_tile[count_roadtype_per_tile['count']>1]

In [244]:
def get_frequent(series):
    return series.value_counts(ascending=False).to_frame().index[0]

tile_to_roadtype_map = df[['Tile_ID','RTTYP']].dropna().groupby('Tile_ID').agg({'RTTYP':[get_frequent]})
tile_to_roadtype_map = tile_to_roadtype_map['RTTYP']['get_frequent'].reset_index()

In [245]:
# df['Tile_ID'].value_counts()
df[df['Tile_ID']==11109]['RTTYP'].value_counts()

M    22
U     2
S     1
Name: RTTYP, dtype: int64

In [246]:
tile_to_roadtype_map.loc[tile_to_roadtype_map['Tile_ID']==4030]['get_frequent'].values[0]
def mapper_val(x):
    temp = tile_to_roadtype_map.loc[tile_to_roadtype_map['Tile_ID']==x]['get_frequent'].values
    if len(temp)>0:
        return temp[0]
    return None
df['RTTYP']=df['Tile_ID'].apply(mapper_val)

In [247]:
# Verifying
df[df['Tile_ID']==10777]['RTTYP'].value_counts()
# df['Tile_ID'].value_counts()

M    17
Name: RTTYP, dtype: int64

In [248]:
df_orig = pd.read_csv('../Data/Tiles_expanded.csv')

In [249]:
df_orig.columns

Index(['Tile_ID', 'Long2', 'Lat2', 'Long1', 'Lat1', 'Mid_lat', 'Mid_long',
       'Stop_Signs', 'Paving_historical', 'Paving_future', 'Bus_stop',
       'Collisions_Future', 'Collisions_Historical'],
      dtype='object')

In [250]:
joined_df = df_orig.join(df,on='Tile_ID',how='inner',lsuffix='_left',rsuffix='_right')

In [251]:
joined_df.columns

Index(['Tile_ID', 'Tile_ID_left', 'Long2', 'Lat2', 'Long1', 'Lat1', 'Mid_lat',
       'Mid_long', 'Stop_Signs', 'Paving_historical', 'Paving_future',
       'Bus_stop', 'Collisions_Future', 'Collisions_Historical', 'wkt_geom',
       'fid', 'LINEARID', 'FULLNAME', 'RTTYP', 'MTFCC', 'Tile_ID_right',
       'left', 'top', 'right', 'bottom'],
      dtype='object')

In [252]:
joined_df = joined_df.drop(['Tile_ID_left','wkt_geom','fid',
                'LINEARID','FULLNAME','Tile_ID_right','left','top','right','bottom','MTFCC'],axis=1)

In [253]:
joined_df['RTTYP'].value_counts(dropna=False)

M    10290
I       82
S       51
U       47
O        1
Name: RTTYP, dtype: int64

In [254]:
joined_df

,Tile_ID,Long2,Lat2,Long1,Lat1,Mid_lat,Mid_long,Stop_Signs,Paving_historical,Paving_future,Bus_stop,Collisions_Future,Collisions_Historical,RTTYP
0,36,-122.514446,37.779636,-122.513306,37.778732,37.779184,-122.513876,0.0,0.0,0.0,0.0,0.0,0.0,M
1,37,-122.514446,37.778732,-122.513306,37.777829,37.778280,-122.513876,0.0,0.0,0.0,0.0,0.0,0.0,M
2,151,-122.513306,37.779636,-122.512166,37.778732,37.779184,-122.512736,0.0,0.0,0.0,0.0,0.0,0.0,M
3,152,-122.513306,37.778732,-122.512166,37.777829,37.778280,-122.512736,0.0,0.0,0.0,0.0,0.0,0.0,M
4,153,-122.513306,37.777829,-122.512166,37.776925,37.777377,-122.512736,0.0,0.0,0.0,0.0,0.0,0.0,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10466,15733,-122.359390,37.728130,-122.358250,37.727227,37.727679,-122.358820,0.0,0.0,0.0,0.0,0.0,0.0,M
10467,15734,-122.359390,37.727227,-122.358250,37.726323,37.726775,-122.358820,0.0,0.0,0.0,0.0,0.0,0.0,M
10468,15735,-122.359390,37.726323,-122.358250,37.725420,37.725871,-122.358820,0.0,0.0,0.0,0.0,0.0,0.0,M
10469,15848,-122.358250,37.728130,-122.357110,37.727227,37.727679,-122.357680,0.0,0.0,0.0,0.0,0.0,0.0,M


In [255]:
!pwd

/Users/abajaj/Documents/UC_Berkeley/GIT/210_Capstone_Aditya_Arisa_Noriel/EDA_notebooks


In [256]:
joined_df.to_csv('/Users/abajaj/Documents/UC_Berkeley/GIT/210_Capstone_Aditya_Arisa_Noriel/Data/Tiles_expanded_road_vector.csv')